In [2]:
from pathlib import Path
import json
from functools import reduce
import math
import datetime as dt
import pytz 
from itertools import product
from collections import OrderedDict
import time
import sys

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.ops as so

import helpers as hp

%load_ext autoreload
%autoreload 2


# Download rent data

In [3]:
base_url = 'http://www.mbie.govt.nz/info-services/housing-property/sector-information-and-statistics/rental-bond-data/detailed/'
file_names = [
    'detailed-lodged-bonds.csv',
    'detailed-mean-rents.csv',
    'detailed-geo-mean-rents.csv',
]

for file_name in file_names:
    url = base_url + file_name
    print("Getting and saving", url)
    r = requests.get(url)
    path = hp.DATA_DIR/'collected'/file_name
    with path.open('w') as tgt:
        tgt.write(r.text)


Getting and saving http://www.mbie.govt.nz/info-services/housing-property/sector-information-and-statistics/rental-bond-data/detailed/detailed-lodged-bonds.csv
Getting and saving http://www.mbie.govt.nz/info-services/housing-property/sector-information-and-statistics/rental-bond-data/detailed/detailed-mean-rents.csv
Getting and saving http://www.mbie.govt.nz/info-services/housing-property/sector-information-and-statistics/rental-bond-data/detailed/detailed-geo-mean-rents.csv


# Prepare rent data

In [4]:
# Reshape and merge all rent data sets

def clean(f, name):
    f = f.copy()
    f = f.rename(columns={
        'SAU': 'au2001',
        'Property_Type': 'property_type',
        'Bedrooms': '#bedrooms'
    })

    # Drop subtotals
    cond = False
    for col in ['au2001', 'property_type', '#bedrooms']:
        cond |= f[col].str.contains('total', case=False)

    f = f[~cond].copy()
    
    # Reshape
    id_vars = ['au2001', 'property_type', '#bedrooms']
    value_vars = [c for c in f.columns if '-' in c]
    f = pd.melt(f, id_vars=id_vars, value_vars=value_vars,
      var_name='quarter', value_name=name)
    
    return f

paths = [
    hp.DATA_DIR/'collected'/'detailed-lodged-bonds.csv',
    hp.DATA_DIR/'collected'/'detailed-mean-rents.csv',
    hp.DATA_DIR/'collected'/'detailed-geo-mean-rents.csv',
]
names = [
    'rent_count', 
    'rent_mean', 
    'rent_geo_mean',
]
frames = []
for path, name in zip(paths, names):
    f = pd.read_csv(path, dtype={'SAU': str})
    frames.append(clean(f, name))
    
f = reduce(lambda x, y: pd.merge(x, y), frames)

# Merge in region data
path = hp.get_path('au2001_csv')
g = pd.read_csv(path, dtype={'au2001': str})
f = f.merge(g)

# Write to file
path = hp.get_path('rents')
f.to_csv(path, index=False)
f[f['rent_count'].notnull()].head()


,au2001,property_type,#bedrooms,quarter,rent_count,rent_mean,rent_geo_mean,au_name,territory,region,rental_area
240,500202,Flat or Apartment,1,1995-03-01,8.0,82.0,82.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo
260,500202,Flat or Apartment,1,1996-03-01,5.0,79.0,79.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo
263,500202,House,3,1996-03-01,5.0,123.0,118.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo
288,500202,House,3,1997-06-01,5.0,154.0,152.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo
313,500202,House,3,1998-09-01,7.0,139.0,137.0,Mangonui East,Far North District,Northland,Mangonui/Kaeo


In [5]:
# Print latest 2 quarters
hp.get_latest_quarters(2)

['2017-09-01', '2017-12-01']

# Explore rents

In [6]:
rents = hp.get_data('rents')
print(rents['quarter'].unique())
rents.head()


['1993-03-01' '1993-06-01' '1993-09-01' '1993-12-01' '1994-03-01'
 '1994-06-01' '1994-09-01' '1994-12-01' '1995-03-01' '1995-06-01'
 '1995-09-01' '1995-12-01' '1996-03-01' '1996-06-01' '1996-09-01'
 '1996-12-01' '1997-03-01' '1997-06-01' '1997-09-01' '1997-12-01'
 '1998-03-01' '1998-06-01' '1998-09-01' '1998-12-01' '1999-03-01'
 '1999-06-01' '1999-09-01' '1999-12-01' '2000-03-01' '2000-06-01'
 '2000-09-01' '2000-12-01' '2001-03-01' '2001-06-01' '2001-09-01'
 '2001-12-01' '2002-03-01' '2002-06-01' '2002-09-01' '2002-12-01'
 '2003-03-01' '2003-06-01' '2003-09-01' '2003-12-01' '2004-03-01'
 '2004-06-01' '2004-09-01' '2004-12-01' '2005-03-01' '2005-06-01'
 '2005-09-01' '2005-12-01' '2006-03-01' '2006-06-01' '2006-09-01'
 '2006-12-01' '2007-03-01' '2007-06-01' '2007-09-01' '2007-12-01'
 '2008-03-01' '2008-06-01' '2008-09-01' '2008-12-01' '2009-03-01'
 '2009-06-01' '2009-09-01' '2009-12-01' '2010-03-01' '2010-06-01'
 '2010-09-01' '2010-12-01' '2011-03-01' '2011-06-01' '2011-09-01'
 '2011-12-

,au2001,property_type,#bedrooms,quarter,rent_count,rent_mean,rent_geo_mean,au_name,territory,region,rental_area
0,500100,House,2,1993-03-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
1,500100,House,3,1993-03-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
2,500100,House,2,1993-06-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
3,500100,House,3,1993-06-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
4,500100,House,2,1993-09-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North


In [13]:
# Slice in time and aggregate 
agg_rents = hp.aggregate_rents(rents, '2017-03-01')
agg_rents.head()

/home/araichev/affordability_nz/py/helpers.py:190: RuntimeWarning: invalid value encountered in double_scalars
  


,rental_area,#bedrooms,region,rent_count,rent_geo_mean,rent_mean,territory
0,Addington,1,Canterbury,104.0,212.765427,215.259615,Christchurch City
1,Addington,2,Canterbury,53.0,330.895107,335.018868,Christchurch City
2,Addington,3,Canterbury,41.0,408.094003,422.170732,Christchurch City
3,Addington,4,Canterbury,0.0,NaN,NaN,Christchurch City
4,Addington,5+,Canterbury,0.0,NaN,NaN,Christchurch City


In [16]:
f = hp.aggregate_rents(rents, date, groupby_cols=('au2001', '#bedrooms'))
cond = f['region'] == 'Auckland'
f[cond]

/home/araichev/affordability_nz/py/helpers.py:190: RuntimeWarning: invalid value encountered in double_scalars
  


,au2001,#bedrooms,region,rent_count,rent_geo_mean,rent_mean,territory
286,505300,1,Auckland,0.0,NaN,NaN,Rodney District
287,505300,2,Auckland,11.0,370.512838,373.636364,Rodney District
288,505300,3,Auckland,13.0,406.852994,409.692308,Rodney District
289,505300,4,Auckland,0.0,NaN,NaN,Rodney District
290,505300,5+,Auckland,0.0,NaN,NaN,Rodney District
291,505400,1,Auckland,0.0,NaN,NaN,Rodney District
292,505400,2,Auckland,0.0,NaN,NaN,Rodney District
293,505400,3,Auckland,0.0,NaN,NaN,Rodney District
294,505400,4,Auckland,0.0,NaN,NaN,Rodney District
295,505500,1,Auckland,5.0,290.000000,295.000000,Rodney District


In [18]:
# What fraction of rental data do we have by #bedrooms?

def hits(group):
    d = {}
    d['hit_frac'] = group['rent_mean'].count()/group['rent_mean'].shape[0]
    return pd.Series(d)

date = '2017-03-01'
f = hp.aggregate_rents(rents, date, groupby_cols=('au2001', '#bedrooms'))
cond = f['region'] == 'Auckland'
print('census area units')
print(f[cond].copy().groupby('#bedrooms').apply(hits).reset_index())

f = hp.aggregate_rents(rents, date, groupby_cols=('rental_area', '#bedrooms'))
cond = f['region'] == 'Auckland'
print('rental area units')
print(f[cond].copy().groupby('#bedrooms').apply(hits).reset_index())


/home/araichev/affordability_nz/py/helpers.py:190: RuntimeWarning: invalid value encountered in double_scalars
  


census area units
  #bedrooms  hit_frac
0         1  0.317757
1         2  0.590116
2         3  0.797721
3         4  0.432277
4        5+  0.079872
rental area units
  #bedrooms  hit_frac
0         1  0.676768
1         2  0.929293
2         3  1.000000
3         4  0.787879
4        5+  0.151515


# Select latest two quarters and slice into regional chunks

In [19]:
# Get latest two quarters of rents
start_date = hp.get_latest_quarters(2)[0]
rents = hp.get_data('rents')
rents = rents[rents['quarter'] >= start_date].copy()

# Create regional slices
for region in hp.REGIONS:
    # Build rents
    region_c = region.capitalize()
    region_rents = rents[rents['region'] == region_c].copy()
    print(region, region_rents['quarter'].unique(), '#rows =', region_rents.shape[0])
    path = hp.get_path('rents', region)
    print('  Saving to', path)
    
    # Create region directory if it does not exist
    if not path.parent.exists():
        path.parent.mkdir()
        
    region_rents.to_csv(path, index=False)
    

auckland ['2017-09-01' '2017-12-01'] #rows = 5472
  Saving to /home/araichev/affordability_nz/data/processed/auckland/rents.csv
canterbury ['2017-09-01' '2017-12-01'] #rows = 2512
  Saving to /home/araichev/affordability_nz/data/processed/canterbury/rents.csv
wellington ['2017-09-01' '2017-12-01'] #rows = 2410
  Saving to /home/araichev/affordability_nz/data/processed/wellington/rents.csv


# JSONize regional rents for the web, grouping by rental area and number of bedrooms

In [20]:
for region in hp.REGIONS:
    region_rents = hp.get_data('rents', region)
    d = hp.build_json_rents(region_rents)
    path = hp.get_path('rents_json', region)
    print('**', region, '\n', d)
    with path.open('w') as tgt:
        json.dump(d, tgt)

    

/home/araichev/affordability_nz/py/helpers.py:190: RuntimeWarning: invalid value encountered in double_scalars
  


** auckland 
 {'New Lynn': {'1': 315.0, '2': 422.0, '4': 617.0, '3': 523.0}, 'Mt Wellington North': {'1': 351.0, '2': 434.0, '4': 760.0, '3': 559.0}, 'Epsom': {'1': 279.0, '2': 506.0, '4': None, '3': 729.0}, 'Botony Downs': {'1': None, '2': None, '4': 673.0, '3': 586.0}, 'Te Atatu Peninsula': {'1': None, '2': 441.0, '4': 643.0, '3': 521.0}, 'Wellsford/Warkworth/Helensville': {'1': 290.0, '2': 402.0, '4': 548.0, '3': 467.0}, 'West Harbour': {'1': None, '2': 432.0, '4': 721.0, '3': 561.0}, 'Kohimarama': {'1': None, '2': 565.0, '4': 1175.0, '3': 743.0}, 'Titirangi': {'1': None, '2': 440.0, '4': 703.0, '3': 526.0}, 'Mt Albert/Owairaka': {'1': 324.0, '2': 445.0, '4': 736.0, '3': 595.0}, 'Takapuna': {'1': 487.0, '2': 601.0, '4': 923.0, '3': 739.0}, 'Ellerslie': {'1': 361.0, '2': 470.0, '4': 808.0, '3': 626.0}, 'Dannemora': {'1': None, '2': 503.0, '4': 690.0, '3': 620.0}, 'Blockhouse Bay/New Windsor': {'1': 302.0, '2': 444.0, '4': 634.0, '3': 557.0}, 'Onehunga/Oranga/Te Papapa': {'1': 226.0, 

** wellington 
 {'Island Bay/Melrose': {'1': 248.0, '2': 460.0, '4': None, '3': 610.0}, 'Petone/Esplanade': {'1': 248.0, '2': 426.0, '4': None, '3': 564.0}, 'Masterton': {'1': 183.0, '2': 257.0, '4': 397.0, '3': 321.0}, 'Kingston/Happy Valley': {'1': 298.0, '2': None, '4': None, '3': 543.0}, 'Titahi Bay/Onepoto/Elsdon': {'1': 234.0, '2': None, '4': None, '3': 419.0}, 'Oriental Bay/Seatoun': {'1': 361.0, '2': 642.0, '4': None, '3': 735.0}, 'Carterton/South Wairarapa': {'1': None, '2': 260.0, '4': None, '3': 333.0}, 'Paramata/Mana/Pukerua Bay': {'1': 273.0, '2': None, '4': None, '3': 534.0}, 'Hataitai': {'1': 255.0, '2': 444.0, '4': 691.0, '3': 566.0}, 'Mt Victoria/Roseneath': {'1': 324.0, '2': 521.0, '4': 870.0, '3': 618.0}, 'Taita/Naenae': {'1': 218.0, '2': 347.0, '4': None, '3': 421.0}, 'Waikanae/Otaki': {'1': None, '2': 335.0, '4': None, '3': 391.0}, 'Trentham West/Eldersley/Clouston Park': {'1': 167.0, '2': 305.0, '4': None, '3': 450.0}, 'Paraparaumu/Raumati': {'1': 192.0, '2': 362.